In [1]:
#MAKE IMPORTS

import numpy as np
import pandas as pd
import cv2
import geoio

In [2]:
class PointTransposer():
    
    def __init__(self, map_p, pic_p, points_p, previous_secs, sub_size, point_pairs=np.array([False])):
        ''' Constructor
        
        Parameters
        ----------
        map_p: str
            path to the large map (a geotiff) you want the points to be put onto
        pic_p: str
            path to slide from video of animals that we are using to create transformation matrix
        points_p: str
            path to list of animal locations (in pixels) from video
        previous_secs: int
            number of previous seconds that the drone was recording video before video of interest started
        sub_size: int
            diameter of the subsection square of the large map that you want to examine to compare to video slides
        point_pairs: numpy array
            optional list of corresponding points between the two pictures that have been identified manually
            each pair of points has [[first picture point], [second picture point]]
            ex: [[[1,2], [3,4]],
                 [[5,6], [7,8]]
                 [[9,10], [11,12]]] '''
        
        #read in geotiff
        self.geo_img = geoio.GeoImage(map_p)

        #read in points locating where animals are in video
        self.csv = False #keeps track of if points come in .csv format or .npy format
        if 'csv' in points_p:
            self.points_file = pd.read_csv(points_p)
            self.points_file = self.points_file.values
            self.csv = True
        elif 'npy' in points_p:
            self.points_file = np.load(points_p)
        else:
            print('not recognized format for file containing animal points.')
                
        #read in pictures in greyscale and load other variables
        self.xPixel, self.yPixel = 30000, 10000        
        full_region = self.get_tiff_pic(map_p)
        surround_region = full_region[int(self.yPixel-sub_size/2):int(self.yPixel+sub_size/2), 
                                      int(self.xPixel-sub_size/2):int(self.xPixel+sub_size/2)]
        specific_pic = cv2.imread(pic_p,0)
        self.pic_height = np.size(specific_pic,0)
        self.sub_size = sub_size

        #make transformation matrix to convert points from video image to points in terms of pixels in large map
        self.matrix = self.get_transformation_matrix(specific_pic, surround_region, point_pairs)
    
    
    def get_slide_points_npy(self, points_file, slide_number, pic_height):
        ''' Gets an array of points marking each animal corresponding to the picture being used

        Parameters
        ----------
        points_path: str
            path to .npy file on computer containing arrays of points locating the animals on each slide of video
        slide_number: int
            number of slides after first slide in video being used (starting at 0 for first slide)
        pic_height: int
            number of rows of pixels in each picture of the animals in video

        Returns
        -------
        np.float32 containing a list of tuples with each point value in pixels '''
        
        #points of one slide - slide_number references csv data, there are 3 tracks and 6 frames for every csv data point
        slide_points = points_file[slide_number]

        #adjust points to make them correct due to weird formatting of input
        adjusted_points = np.zeros(slide_points.shape)
        adjusted_points = [[point[1],pic_height - point[0]]  for point in slide_points]

        return np.float32([adjusted_points])
    
    
    def get_slide_points_csv(self, points_file, slide_number, pic_height):
        ''' Gets an array of points marking each animal corresponding to the picture being used

        Parameters
        ----------
        points_file: 
            path to .npy file on computer containing arrays of points locating the animals on each slide of video
        slide_number: int
            number of slides after first slide in video being used (starting at 0 for first slide)
        pic_height: int
            number of rows of pixels in each picture of the animals in video

        Returns
        -------
        np.float32 containing a list of tuples with each point value in pixels '''
        
        #points of one slide - slide_number references csv data, there are 3 tracks and 6 frames for every csv data point
        slide_points = points_file[slide_number]
        if slide_number == 0:
            slide_points = points_file[1]

        #adjust points to make them correct due to weird formatting of input
        adjusted_points = np.zeros((int((len(slide_points) - 1)/2),2))

        for i in range(int((len(slide_points)-1)/2)):
            adjusted_points[i] = [slide_points[2*i+1], pic_height - slide_points[2+2*i]]

        return np.float32([adjusted_points])

    
    def get_track_points_csv(self, points_file, track_number, pic_height): #may need more editing
        ''' Gets an array of points marking each animal corresponding to the picture being used

        Parameters
        ----------
        points_path: str
            path to .npy file on computer containing arrays of points locating the animals on each slide of video
        track_number: int
            number of track you want to examine
        pic_height: int
            number of rows of pixels in each picture of the animals in video

        Returns
        -------
        np.float32 containing a list of tuples with each point value in pixels '''
        
        #points of one slide - slide_number references csv data, there are 3 tracks and 6 frames for every csv data point
        track_points = points_file[:, track_number*2:track_number*2+2]

        print(track_points)
        
        #adjust points to make them correct due to weird formatting of input
        adjusted_points = np.zeros(track_points.shape)
        for i in range(len(track_points)):
            adjusted_points[i] = [track_points[i][0], pic_height - track_points[i][1]]

        return np.float32([adjusted_points])


    def get_starting_point(self, csv_file, previous_seconds):
        ''' Get the row number of the first data point from csv file that should be used, since there are data points in
        the csv file that are irrelevant to the data we care about

        Parameters
        ----------
        csv_file: str
            path to the csv file with latitude/longitude and other data points recorded many times each second that the
            drone was in the air
        previous_seconds: int
            the number of previous seconds the drone had been in the air recording before it started recording data we
            care about (can be found in other csv file with info about all drone flights)

        Returns
        -------
        int: the first row number in csv file that we should use '''

        DATA_POINTS_PER_SEC = 10 #number of data points in csv file taken per second

        #get column for the 'isVideo' data column in csv file
        isVideo_col = csv_file['isVideo']

        #set video_start to when isVideo first turns to 1
        video_start = 0
        for i in range(0, len(isVideo_col)):
            if isVideo_col[i] == 1:
                video_start = i
                break
            if i >= len(isVideo_col) - 1:
                print("warning: isVideo column in csv file does not have any '1' values")

        #return starting frame
        return previous_seconds*DATA_POINTS_PER_SEC + video_start


    def get_center_point(self, csv_file, starting_point, data_point_number, geo_img, coord_sys1=4326, coord_sys2=32637):
        ''' Returns latitude and longitude point of center of specified video frame

        Parameters
        ----------
        csv_file: str
            path to the csv file with latitude/longitude and other data points recorded many times each second that the
            drone was in the air
        starting_point: int
            the first row number of csv file that contains data relevant to our video
        data_point-number: int
            the number of rows below starting_point where our data is
        geo_img: GeoImage (?)
            the geotiff image having been read in using geoio.GeoImage
        coord_sys1: int
            EPSG number of the coordinate system the lat/lon coordinates from csv file come in - csv is EPSG4326
        coord_sys2: int
            EPSG number of the coordinate system the lat/lon coordinates of map are in - geotiff is EPSG32637

        Returns
        -------
        array: [xPixel, yPixel] '''
         
        from pyproj import Proj, transform

        #get latitude and longitude
        lat, lon = csv_file.loc[starting_point+data_point_number, 'latitude':'longitude'] #returns (lat, lon)

        #return coordinates in correct system
        inProj = Proj(init='epsg:'+str(coord_sys1))
        outProj = Proj(init='epsg:'+str(coord_sys2))
        lon, lat = transform(inProj,outProj,lon,lat)

        #convert to pixels
        return geo_img.proj_to_raster(lon, lat)


    def get_tiff_pic(self, map_path):
        ''' Get greyscale image from geotiff file

        Parameters
        ----------
        map_path: str
            path to geotiff file in computer

        Returns
        -------
        array (?) in same format as cv2.imread() '''
        from osgeo import gdal

        gdal.UseExceptions()

        #return array
        ds = gdal.Open(map_path)
        rb = ds.GetRasterBand(1)
        return rb.ReadAsArray()


    def filter_matches(self, kp1, kp2, matches, ratio = 0.75):
        ''' For aid in getting transformation matrix, filter matches where descriptors aren't close enough

        Parameters
        ----------
        kp1: array
            array of key points in first picture
        kp2: array
            array of key points in second picture
        matches: array
            array of keypoints that are matches
        ratio: float
            used to filter out matches based on how close they match

        Returns
        -------
        np.float32 array: [points from first picture, corresponding points from second picture] '''

        # Sort matches in order of their distance
        matches = sorted(matches, key = lambda x:x[0].distance)

        #filter out matches based on how closely they match and keep top N matches
        good = []
        counter = 0
        N_MATCHES = 15
        for m in matches:
            if len(m) == 2 and m[0].distance < m[1].distance * ratio:
                m = m[0]
                good.append( m )
                counter += 1
            if counter>=N_MATCHES:
                break

        print("Number of matches found = %d" % len(good))

        #convert to correct format
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

        return src_pts, dst_pts


    def get_transformation_matrix(self, specific_pic, full_region, points_list=np.array([False])):
        ''' Returns transformation matrix to convert points from animal footage to corret pixels in full picture

        Parameters
        ----------
        specific_pic: numpy array
            array of greyscale points for animal footage picture
        full_region:  numpy array
            array of greyscale points for subsection of full map that includes at least part of animal footage picture
        points_list: numpy array
            optional list of corresponding points between the two pictures that have been identified manuall
            each pair of point has [[first picture point], [second picture point]]
            ex: [[[1,2], [3,4]],
                 [[5,6], [7,8]]
                 [[9,10], [11,12]]]

        Returns
        -------
        array: 3x3 array of floats as the transformation matrix '''

        print('finding transformation matrix...')

        if points_list.any() == False: #if the person has not sent in their own set of points

            #can adjust parameters and which detector to use to adjust types of detected features
            orb = cv2.ORB_create(nfeatures=100000, scoreType=cv2.ORB_FAST_SCORE)        
            matcher = cv2.BFMatcher(cv2.NORM_L2)

            #detect key points in each image
            kp1, desc1 = orb.detectAndCompute(specific_pic, None)
            kp2, desc2 = orb.detectAndCompute(full_region, None)

            #match key points between images
            raw_matches = matcher.knnMatch(desc1, trainDescriptors = desc2, k = 2) #2
            src_pts, dst_pts = self.filter_matches(kp1, kp2, raw_matches)

            #return mask
            return cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)[0]

        else: #if the person has sent in their own set of points

            #put lists of points from both pictures in correct format
            src_pts = np.float32([ pair[0] for pair in points_list ]).reshape(-1,1,2)
            dst_pts = np.float32([ pair[1] for pair in points_list ]).reshape(-1,1,2)

            #return mask
            return cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)[0]
    
    
    def get_points(self, slide_number, folder, type='slide'):
        ''' Put points from one slide in animal footage into csv file and saves it in required folder
        
        Parameters
        ----------
        slide_number: int
            if type == slide, the time (in 10ths of seconds) that the frame of interest appears since the video started
            if type == track, the number of the track to use
        folder: str
            name of folder to put csv file into
        type: str
            'slide' if you want to get points from one slide, 'track' to get points from a track
        '''
        
        #get xy pixel points identifying animals in video image
        if type == 'slide':
            if self.csv == False:
                pts = self.get_slide_points(self.points_file, slide_number*3, self.pic_height)
            elif self.csv == True:
                pts = self.get_slide_points_csv(self.points_file, slide_number*3, self.pic_height)
        else:
            pts = self.get_track_points_csv(self.points_file, slide_number, self.pic_height)

        #get points of animals in terms of pixels of map subsection
        transformation = cv2.perspectiveTransform(pts,self.matrix)

        #get points in terms of large map
        x_increase = int(self.xPixel - self.sub_size/2)#180 #amount to add to x value to get x in terms of entire map
        y_increase = int(self.yPixel - self.sub_size/2)#145
        map_view_points = np.zeros(transformation[0].shape)
        map_view_points = [(point[0]+x_increase, point[1]+y_increase) for point in transformation[0]]

        #get points in terms of lat/lon values
        lon_lat_final = np.zeros((len(map_view_points), 2))
        lon_lat_final = [(self.geo_img.raster_to_proj(map_view_points[i][0], map_view_points[i][1]))
                         for i in range(0, len(map_view_points))]

        #output values to a csv
        np.savetxt(folder+'/'+type+str(slide_number)+".csv",lon_lat_final,delimiter=",")
    
    
    def get_all_points(self, folder):
        ''' Transform points in csv file into latitude/longitude coordinates, puts csv file into required folder
        
        Parameters
        ----------
        folder: str
            name of folder to put csv file into
        '''
        
        #get all points from csv file
        good_points = np.zeros((int(len(self.points_file) - 1), int((len(self.points_file[0]) - 1)/2),2))
        for i in range(len(self.points_file)-1):
            good_points[i] = self.get_slide_points_csv(self.points_file, i, self.pic_height)
        
        #get transformation of points
        transformation = cv2.perspectiveTransform(good_points, self.matrix)
        
        #get points in terms of larger map
        x_increase = int(self.xPixel - self.sub_size/2)
        y_increase = int(self.yPixel - self.sub_size/2)
        map_view_points = np.zeros(good_points.shape)
        for i in range(len(good_points)):
            map_view_points[i] = [(point[0]+x_increase, point[1]+y_increase) for point in transformation[i]]
        
        #get points in terms of lat/lon values
        lon_lat_final = np.zeros(good_points.shape)
        for i in range(len(good_points)):
            lon_lat_final[i] = [(self.geo_img.raster_to_proj(map_view_points[i][j][0], map_view_points[i][j][1]))
                             for j in range(0, len(map_view_points[i]))]
        
        #create matrix to put lat/lon values in
        positions = np.empty((len(map_view_points), len(map_view_points[0])*2))
        for i in range(0, len(positions)):
            for j in range(0, len(positions[0])):
                if j % 2 == 0:
                    positions[i][j] = lon_lat_final[i][int(j/2)][0]
                else:
                    positions[i][j] = lon_lat_final[i][int(j/2)][1]
        
        #column labels for matrix with lat/lon values
        column_names = []
        for ind in range(len(map_view_points[0])*2):
            xy = ''
            if ind % 2 == 0:
                xy = 'lon'
            else:
                xy = 'lat'
            column_names.append('track_' + str(ind//2) + '_' + xy)
        
        #output values to a csv
        positions_df = pd.DataFrame(positions, columns=column_names)
        positions_df.to_csv(folder+'/all_points.csv')

        print('all_points.csv created')

In [3]:
class Transposer():
    
    def __init__(self, transposer_start, transposer_end, input_length):
        ''' transpose points to latitude/longitude coordinates more accurately than before. Pass in two csv files with 
        lat/lon locations, one based on data from beginning of video, one based on the end of the video. Take weighted 
        averages of point locations to make lat/lon locations of points at the beginning of the video be based more on
        the transformation matrix from the beginning of the video, and the locations at the end based more on the 
        transformation matrix from the end of the video.
        
        input_length = number of latitude/longitude points recorded for drone video
        '''
        self.transposer_start, self.transposer_end = transposer_start, transposer_end
        self.input_length = input_length
    
    def get_points(self, slide_number, folder, type='slide'):
        csv_p = folder+'/'+type+str(slide_number)+".csv"
        fraction = slide_number / self.input_length
        
        #get first points
        self.transposer_start.get_points(slide_number, folder,type=type)
        csv_start = pd.read_csv(csv_p).values
        
        #get last points
        self.transposer_end.get_points(slide_number, folder, type=type)
        csv_end = pd.read_csv(csv_p).values
        
        #take averages
        final_result = np.zeros(csv_start.shape)
        for i in range(len(csv_start)):
            final_result[i] = [csv_start[i][0]*fraction + csv_end[i][0]*(1-fraction), 
                               csv_start[i][1]*fraction + csv_end[i][1]*(1-fraction)]
        
        np.savetxt(csv_p,final_result,delimiter=",")

In [4]:
map_p = '/Users/dbasili/csv_files_airstrip2/Geotiffs/riverside005_011_merge_transparent_mosaic_group1.tif'
pic_p = '/Users/dbasili/csv_files_airstrip2/frames/APR08_2018_C_DJI_0010_2.jpg' #change number when necessary
csv_p = '/Users/dbasili/csv_files_airstrip2/2018-04-08_8-26-33_Standard.csv'
folder = '/Users/dbasili/csv_files_airstrip2'
#points_p = '/Users/dbasili/koger_drive/positions.npy' #points of animals on one slide
points_p = '/Users/dbasili/csv_files_airstrip2/APR08_2018_C_DJI_0010-tracks.csv'
slide_num = 1636                                         #number after starting_number to get data from
previous_secs = 328+328+328+240+328+328+328+317+327 #600 #602 #655   #in csv file before getting to video of interest
subsection_size = 6000                                #size of full map subsection

#[244,874]=newpic, [3452,956]=originalpic
y_1636 = 874-956
x_1636 = 244-3452

point_pairs0 = np.float32([[[1814,548], [640,2821]],
                           [[1605,617], [717,3068]],
                           [[994,1016], [1183,3797]],
                           [[534,938], [1098,4352]],
                           [[245,871], [1014,4727]],
                           [[3071,470], [547,1391]],
                           [[2161,1293], [1499,2415]],
                           [[3639,1269], [1452,713]],
                           [[2940,1434], [1649,1503]],
                           [[3441,1101], [1261,947]]])

point_pairs1636 = np.float32([[[3452+x_1636,954+y_1636],  [1015,4728]],
                              [[1806+x_1636,4996+y_1636], [3224,1663]],
                              [[2300+x_1636,1626+y_1636], [1807,6066]],
                              [[1422+x_1636,867+y_1636],  [934,7071]],
                              [[1584+x_1636,459+y_1636],  [473,6889]],
                              [[2110+x_1636,552+y_1636],  [578,6292]]])

transposer=PointTransposer(map_p,pic_p,points_p,previous_secs,subsection_size,point_pairs=point_pairs0)

finding transformation matrix...


In [5]:
transposer0=PointTransposer(map_p,pic_p,points_p,previous_secs,subsection_size,point_pairs=point_pairs0)
transposer1636=PointTransposer(map_p,pic_p,points_p,previous_secs,subsection_size,point_pairs=point_pairs1636)
video_length = 3272

final_transposer = Transposer(transposer0,transposer1636,video_length)

finding transformation matrix...
finding transformation matrix...


In [6]:
transposer.get_points(0, folder)
transposer.get_points(2419, folder)
transposer.get_points(3272, folder)
transposer.get_points(1, folder,type='track')
transposer.get_points(2, folder,type='track')

[[           nan            nan]
 [  442.65307617  2433.5378418 ]
 [  443.54528809  2433.24023438]
 ...
 [ 1108.29467773 -1734.64013672]
 [ 1108.26623535 -1735.06445312]
 [ 1108.48583984 -1734.66845703]]
[[           nan            nan]
 [  904.56005859  2444.3059082 ]
 [  904.60339355  2443.94555664]
 ...
 [ 1177.88024902 -1817.77270508]
 [ 1178.60742188 -1817.64318848]
 [ 1178.46557617 -1816.58056641]]


In [7]:
final_transposer.get_points(0, folder)
final_transposer.get_points(2419, folder)
final_transposer.get_points(3272, folder)

In [8]:
transposer.get_all_points(folder)

all_points.csv created


In [9]:
print('done')

done
